In [5]:
from binance.client import AsyncClient, HistoricalKlinesType
import pandas as pd

from datetime import datetime, timedelta
import os

import asyncio

In [3]:
client = await AsyncClient.create()

class Binance_Batch_Klines_Downloader:
  def __init__(self, days_back = 2, interval = "1h", klines_types=[HistoricalKlinesType.FUTURES], rate_limit_ps=2):
    self.days_back = days_back
    self.start = str(datetime.utcnow() - timedelta(days = self.days_back))
    
    self.interval = interval
    self.klines_types = klines_types
    self.rate_limit_ps = rate_limit_ps
    
  def _bars_to_df(self, bars):
    df = pd.DataFrame(bars)
    df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
    df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                  "Close Time", "Quote Asset Volume", "Number of Trades",
                  "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
    df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
    df.set_index("Date", inplace = True)
    for column in df.columns:
        df[column] = pd.to_numeric(df[column], errors = "coerce")
        
    return df
  
  async def _get_symbols(self, klines_type):
    exchange_info = None
    if klines_type == HistoricalKlinesType.FUTURES:
      exchange_info = await client.futures_exchange_info()
    if klines_type == HistoricalKlinesType.SPOT:
      exchange_info = await client.get_exchange_info()
    
    if exchange_info is not None:
      return list(map(lambda x: x['symbol'], exchange_info['symbols']))
  
  async def _download_kline_type(self, klines_type):
    symbols = await self._get_symbols(klines_type)
    if not len(symbols):
      return
    
    new_dir = "Binance_Historical_%s_%s_%i_days_%s" % (self.interval, klines_type.name, self.days_back, str(datetime.utcnow().replace(microsecond=0).isoformat()))
    os.mkdir(new_dir)
    
    chunks = []
    for idx, _ in enumerate(symbols):
      if idx % self.rate_limit_ps == 0:
        symbols_chunk = symbols[idx:idx+self.rate_limit_ps]
        futures_chunk = []
        
        for symbol in symbols_chunk:
          futures_chunk.append(client.get_historical_klines(symbol = symbol, interval = self.interval,
                                        start_str = self.start, end_str = None, limit = 1000, klines_type=klines_type))
          
        chunks.append(futures_chunk)
        
    for index, chunk in enumerate(chunks):
      print("Fetching data for ↓ %s %s" % (klines_type.name, self.interval), "%i / %i" % (index, len(chunks)))
      results = await asyncio.gather(*chunk)
      
      for i, bars in enumerate(results):
        print(symbols[index*self.rate_limit_ps+i])
        try:
          df = self._bars_to_df(bars)
          df.to_csv("%s/%s_%s_%s.csv" % (new_dir, symbols[index*self.rate_limit_ps+i], klines_type.name, self.interval))
        except:
          # print("raw: ", bars)
          print("Couldn't construct DataFrame from raw data for %s" % symbols[index*self.rate_limit_ps+i])
          
  async def download(self):
    for type in self.klines_types:
      print(type)
      await self._download_kline_type(type)

<h2 style="color: orange">DEMO</h2>

In [ ]:
downloader = Binance_Batch_Klines_Downloader(days_back=20, interval="1h", klines_types=[HistoricalKlinesType.FUTURES], rate_limit_ps=7)

In [ ]:
await downloader.download()